In [12]:
import pandas as pd
import numpy as np

import re
import nltk.data
from nltk.corpus import stopwords

from bs4 import BeautifulSoup
import logging

from gensim.models import word2vec, Word2Vec, KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

from tqdm import tqdm

import pickle

import xgboost as xgb
from xgboost.sklearn import XGBRegressor

from sklearn.metrics import mean_squared_error as mse

In [2]:
reviews = pd.read_csv('all_reviews.csv')
with open("all_cleaned_reviews.txt", "rb") as fp:   # Unpickling
    fp.seek(0)
    all_cleaned_reviews = pickle.load(fp)
#all_cleaned_reviews = pickle.load('all_cleaned_reviews.txt')

In [3]:
len(all_cleaned_reviews)

3288164

In [10]:
glove_input_file = '../../../glove.840B.300d.txt'
word2vec_output_file = 'word2vec_from_glove.txt.word2vec'
glove2word2vec(glove_input_file, word2vec_output_file)

(2196017, 300)

In [13]:
#with open("word2vec_from_glove.txt", "rb") as fp:   # Unpickling
    #fp.seek(0)
    #fp.
    #model = Word2Vec.load(fp)
model = KeyedVectors.load_word2vec_format('word2vec_from_glove.txt.word2vec', binary=False)

In [14]:
num_features = 300
def review_to_wordlist(review, remove_stopwords):
    return review.split()

In [15]:
index2word_set = set(model.wv.index2word)
def makeFeatureVec(words, model, num_features):
    featureVec = np.zeros((num_features,), dtype="float32")
    nwords = 0.0
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec, model[word])
    featureVec = np.divide(featureVec,nwords)
    return featureVec

C:\Users\I345351\AppData\Local\Continuum\anaconda2\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


In [16]:
def getAvgFeatureVecs(reviews, model, num_features):
    counter = 0
    reviewFeatureVecs = np.zeros((len(reviews), num_features), dtype="float32")
    for review in tqdm(reviews):
        reviewFeatureVecs[counter] = makeFeatureVec(review, model, num_features)
        counter = counter + 1
    return reviewFeatureVecs

In [17]:
clean_train_reviews = []
for review in tqdm(all_cleaned_reviews):
    clean_train_reviews.append(review_to_wordlist(review, remove_stopwords=True))

100%|██████████| 3288164/3288164 [01:05<00:00, 49984.25it/s]


In [18]:
listing_ids = reviews['listing_id']
del reviews

In [19]:
trainDataVecs = getAvgFeatureVecs(clean_train_reviews, model, num_features)

  0%|          | 1639/3288164 [00:02<1:34:24, 580.18it/s]C:\Users\I345351\AppData\Local\Continuum\anaconda2\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in divide
  if __name__ == '__main__':
100%|██████████| 3288164/3288164 [09:12<00:00, 5951.83it/s]


In [20]:
train_data_vecs_df = pd.DataFrame(data=trainDataVecs, columns=['w2v_'+str(x) for x in range(300)])

In [21]:
del trainDataVecs

In [22]:
train_data_vecs_df.head()

,w2v_0,w2v_1,w2v_2,w2v_3,w2v_4,w2v_5,w2v_6,w2v_7,w2v_8,w2v_9,...,w2v_290,w2v_291,w2v_292,w2v_293,w2v_294,w2v_295,w2v_296,w2v_297,w2v_298,w2v_299
0,-0.007500,0.053015,-0.030284,-0.085912,0.021917,0.075316,-0.374672,-0.087778,0.251036,1.747512,...,-0.258446,-0.016723,-0.102804,-0.161646,-0.058546,-0.077236,0.058416,-0.228278,-0.169900,0.031526
1,0.110486,0.118191,-0.127485,-0.049677,0.183837,0.047695,0.016938,-0.203206,0.064499,1.988351,...,-0.102617,0.104788,-0.076123,0.051388,0.068400,0.004319,0.049317,-0.105074,-0.014143,0.046933
2,0.091432,0.225427,-0.182954,-0.008398,0.077003,0.001038,0.063045,-0.116057,-0.024711,2.238188,...,-0.143708,0.055845,-0.106815,-0.013823,0.098859,0.046101,-0.090006,-0.092447,0.058248,0.135792
3,0.085635,0.113545,-0.188197,-0.067804,0.061259,0.142478,0.015654,-0.132785,0.087651,1.784935,...,-0.129186,0.166398,0.095126,0.067250,0.032327,0.010454,0.067277,-0.133682,0.015929,0.242948
4,0.005741,0.199449,-0.196606,-0.039012,0.184463,0.025837,0.034836,-0.197611,0.062912,2.034414,...,-0.121703,0.089959,-0.070807,0.062765,0.095825,-0.049051,0.039055,-0.068176,0.069366,0.103268


In [23]:
train_data_vecs_df['listing_id'] = listing_ids

In [24]:
['w2v_' + str(x) for x in range(2)]

['w2v_0', 'w2v_1']

In [25]:
#cols = ['w2v_'+str(x) for x in range(300)]
temp = train_data_vecs_df.groupby('listing_id')['w2v_0', 'w2v_1', 'w2v_2', 'w2v_3', 'w2v_4'].mean().reset_index()
#temp['listing_id'] = listing_ids
for i in tqdm(range(5, 296, 5)):
    col = ['w2v_' + str(x) for x in range(i, i+5)]
    temp = temp.merge(train_data_vecs_df.groupby('listing_id')[col].mean().reset_index(), on='listing_id', how='left')#, 'w2v_2', 'w2v_3', 'w2v_4', 'w2v_5', 'w2v_6', 'w2v_7', 'w2v_8', 'w2v_9', 'w2v_10', 'w2v_11', 'w2v_12', 'w2v_13', 'w2v_14', 'w2v_15', 'w2v_16', 'w2v_17', 'w2v_18', 'w2v_19', 'w2v_20', 'w2v_21', 'w2v_22', 'w2v_23', 'w2v_24', 'w2v_25', 'w2v_26', 'w2v_27', 'w2v_28', 'w2v_29', 'w2v_30', 'w2v_31', 'w2v_32', 'w2v_33', 'w2v_34', 'w2v_35', 'w2v_36', 'w2v_37', 'w2v_38', 'w2v_39', 'w2v_40', 'w2v_41', 'w2v_42', 'w2v_43', 'w2v_44', 'w2v_45', 'w2v_46', 'w2v_47', 'w2v_48', 'w2v_49', 'w2v_50'].mean().reset_index()#, 'w2v_51', 'w2v_52', 'w2v_53', 'w2v_54', 'w2v_55', 'w2v_56', 'w2v_57', 'w2v_58', 'w2v_59', 'w2v_60', 'w2v_61', 'w2v_62', 'w2v_63', 'w2v_64', 'w2v_65', 'w2v_66', 'w2v_67', 'w2v_68', 'w2v_69', 'w2v_70', 'w2v_71', 'w2v_72', 'w2v_73', 'w2v_74', 'w2v_75', 'w2v_76', 'w2v_77', 'w2v_78', 'w2v_79', 'w2v_80', 'w2v_81', 'w2v_82', 'w2v_83', 'w2v_84', 'w2v_85', 'w2v_86', 'w2v_87', 'w2v_88', 'w2v_89', 'w2v_90', 'w2v_91', 'w2v_92', 'w2v_93', 'w2v_94', 'w2v_95', 'w2v_96', 'w2v_97', 'w2v_98', 'w2v_99'].mean().reset_index()#, 'w2v_100', 'w2v_101', 'w2v_102', 'w2v_103', 'w2v_104', 'w2v_105', 'w2v_106', 'w2v_107', 'w2v_108', 'w2v_109', 'w2v_110', 'w2v_111', 'w2v_112', 'w2v_113', 'w2v_114', 'w2v_115', 'w2v_116', 'w2v_117', 'w2v_118', 'w2v_119', 'w2v_120', 'w2v_121', 'w2v_122', 'w2v_123', 'w2v_124', 'w2v_125', 'w2v_126', 'w2v_127', 'w2v_128', 'w2v_129', 'w2v_130', 'w2v_131', 'w2v_132', 'w2v_133', 'w2v_134', 'w2v_135', 'w2v_136', 'w2v_137', 'w2v_138', 'w2v_139', 'w2v_140', 'w2v_141', 'w2v_142', 'w2v_143', 'w2v_144', 'w2v_145', 'w2v_146', 'w2v_147', 'w2v_148', 'w2v_149', 'w2v_150', 'w2v_151', 'w2v_152', 'w2v_153', 'w2v_154', 'w2v_155', 'w2v_156', 'w2v_157', 'w2v_158', 'w2v_159', 'w2v_160', 'w2v_161', 'w2v_162', 'w2v_163', 'w2v_164', 'w2v_165', 'w2v_166', 'w2v_167', 'w2v_168', 'w2v_169', 'w2v_170', 'w2v_171', 'w2v_172', 'w2v_173', 'w2v_174', 'w2v_175', 'w2v_176', 'w2v_177', 'w2v_178', 'w2v_179', 'w2v_180', 'w2v_181', 'w2v_182', 'w2v_183', 'w2v_184', 'w2v_185', 'w2v_186', 'w2v_187', 'w2v_188', 'w2v_189', 'w2v_190', 'w2v_191', 'w2v_192', 'w2v_193', 'w2v_194', 'w2v_195', 'w2v_196', 'w2v_197', 'w2v_198', 'w2v_199', 'w2v_200', 'w2v_201', 'w2v_202', 'w2v_203', 'w2v_204', 'w2v_205', 'w2v_206', 'w2v_207', 'w2v_208', 'w2v_209', 'w2v_210', 'w2v_211', 'w2v_212', 'w2v_213', 'w2v_214', 'w2v_215', 'w2v_216', 'w2v_217', 'w2v_218', 'w2v_219', 'w2v_220', 'w2v_221', 'w2v_222', 'w2v_223', 'w2v_224', 'w2v_225', 'w2v_226', 'w2v_227', 'w2v_228', 'w2v_229', 'w2v_230', 'w2v_231', 'w2v_232', 'w2v_233', 'w2v_234', 'w2v_235', 'w2v_236', 'w2v_237', 'w2v_238', 'w2v_239', 'w2v_240', 'w2v_241', 'w2v_242', 'w2v_243', 'w2v_244', 'w2v_245', 'w2v_246', 'w2v_247', 'w2v_248', 'w2v_249', 'w2v_250', 'w2v_251', 'w2v_252', 'w2v_253', 'w2v_254', 'w2v_255', 'w2v_256', 'w2v_257', 'w2v_258', 'w2v_259', 'w2v_260', 'w2v_261', 'w2v_262', 'w2v_263', 'w2v_264', 'w2v_265', 'w2v_266', 'w2v_267', 'w2v_268', 'w2v_269', 'w2v_270', 'w2v_271', 'w2v_272', 'w2v_273', 'w2v_274', 'w2v_275', 'w2v_276', 'w2v_277', 'w2v_278', 'w2v_279', 'w2v_280', 'w2v_281', 'w2v_282', 'w2v_283', 'w2v_284', 'w2v_285', 'w2v_286', 'w2v_287', 'w2v_288', 'w2v_289', 'w2v_290', 'w2v_291', 'w2v_292', 'w2v_293', 'w2v_294', 'w2v_295', 'w2v_296', 'w2v_297', 'w2v_298', 'w2v_299'].mean().reset_index()

100%|██████████| 59/59 [00:38<00:00,  1.53it/s]


In [26]:
temp.head()

,listing_id,w2v_0,w2v_1,w2v_2,w2v_3,w2v_4,w2v_5,w2v_6,w2v_7,w2v_8,...,w2v_290,w2v_291,w2v_292,w2v_293,w2v_294,w2v_295,w2v_296,w2v_297,w2v_298,w2v_299
0,6,0.104832,0.135290,-0.136698,-0.077068,0.150568,0.013708,0.017012,-0.173440,0.002207,...,-0.096481,0.057657,-0.015273,0.041585,0.152477,0.055443,0.038041,-0.079013,0.021239,0.126959
1,109,0.257166,0.156430,-0.067845,-0.115914,0.090169,-0.006481,0.009655,-0.105731,-0.016771,...,-0.017028,0.187695,-0.021072,0.004881,0.062357,-0.090380,-0.035381,0.012428,0.016011,0.095739
2,177,0.072598,0.153454,-0.181378,-0.046265,0.097177,0.036257,0.029202,-0.212942,0.032734,...,-0.148288,0.087071,-0.006015,0.039476,0.107237,0.061776,0.029971,-0.082056,0.042877,0.090477
3,344,-0.038872,0.187163,-0.148552,-0.007992,-0.031216,0.056267,0.071489,-0.242644,0.026321,...,-0.081829,0.108793,-0.014770,0.091156,0.148940,-0.098985,-0.089874,-0.086044,0.019668,0.110363
4,590,0.090016,0.150616,-0.149289,-0.044620,0.112512,0.034156,0.053903,-0.196740,0.021123,...,-0.083770,0.067124,-0.044315,0.048432,0.143617,0.012343,-0.032879,-0.089097,-0.006444,0.088553


In [27]:
temp.to_csv('w2v_features_glove_300.csv', index=False)

In [38]:
for c in ['w2v_'+str(x) for x in range(300)]:
    print '\'' + c + '\',',

'w2v_0', 'w2v_1', 'w2v_2', 'w2v_3', 'w2v_4', 'w2v_5', 'w2v_6', 'w2v_7', 'w2v_8', 'w2v_9', 'w2v_10', 'w2v_11', 'w2v_12', 'w2v_13', 'w2v_14', 'w2v_15', 'w2v_16', 'w2v_17', 'w2v_18', 'w2v_19', 'w2v_20', 'w2v_21', 'w2v_22', 'w2v_23', 'w2v_24', 'w2v_25', 'w2v_26', 'w2v_27', 'w2v_28', 'w2v_29', 'w2v_30', 'w2v_31', 'w2v_32', 'w2v_33', 'w2v_34', 'w2v_35', 'w2v_36', 'w2v_37', 'w2v_38', 'w2v_39', 'w2v_40', 'w2v_41', 'w2v_42', 'w2v_43', 'w2v_44', 'w2v_45', 'w2v_46', 'w2v_47', 'w2v_48', 'w2v_49', 'w2v_50', 'w2v_51', 'w2v_52', 'w2v_53', 'w2v_54', 'w2v_55', 'w2v_56', 'w2v_57', 'w2v_58', 'w2v_59', 'w2v_60', 'w2v_61', 'w2v_62', 'w2v_63', 'w2v_64', 'w2v_65', 'w2v_66', 'w2v_67', 'w2v_68', 'w2v_69', 'w2v_70', 'w2v_71', 'w2v_72', 'w2v_73', 'w2v_74', 'w2v_75', 'w2v_76', 'w2v_77', 'w2v_78', 'w2v_79', 'w2v_80', 'w2v_81', 'w2v_82', 'w2v_83', 'w2v_84', 'w2v_85', 'w2v_86', 'w2v_87', 'w2v_88', 'w2v_89', 'w2v_90', 'w2v_91', 'w2v_92', 'w2v_93', 'w2v_94', 'w2v_95', 'w2v_96', 'w2v_97', 'w2v_98', 'w2v_99', 'w2v_100',

In [36]:
train = pd.read_csv('train_sent.csv')
test = pd.read_csv('test_sent.csv')

In [37]:
train = train.merge(temp, on='listing_id', how='left')
test = test.merge(temp, on='listing_id', how='left')

In [38]:
train.shape

(70894, 312)

In [48]:
test.head()

,listing_id,compound,neg,neu,pos,w2v_0,w2v_1,w2v_2,w2v_3,w2v_4,...,w2v_290,w2v_291,w2v_292,w2v_293,w2v_294,w2v_295,w2v_296,w2v_297,w2v_298,w2v_299
0,177,0.789695,0.020895,0.655263,0.323816,0.125443,0.099148,0.239083,-0.109373,0.115686,...,0.343390,0.079103,0.099657,-0.108110,0.214957,0.185194,-0.007544,-0.052955,0.061775,-0.084681
1,590,0.887377,0.010110,0.679164,0.308788,0.003294,0.097149,0.235431,-0.051583,0.079655,...,0.371077,0.131271,-0.067424,0.024969,0.142040,0.190251,-0.029581,-0.054145,0.055785,-0.112137
2,958,0.821159,0.014551,0.691268,0.286315,0.215096,0.095965,0.287974,-0.119683,0.054009,...,0.298860,-0.070695,-0.036481,-0.167621,0.161097,0.143639,0.066764,0.019375,-0.000382,-0.147448
3,1788,0.886797,0.002567,0.695867,0.301567,0.229944,0.084868,0.307960,-0.106768,0.125780,...,0.353351,0.056552,0.017254,-0.058394,0.197951,0.115530,0.090405,-0.017649,0.054126,-0.167841
4,2384,0.909629,0.011200,0.706920,0.281907,0.109573,0.115508,0.294353,-0.011445,0.069103,...,0.351657,0.046355,-0.075938,0.044893,0.171532,0.197407,0.021918,-0.060119,0.025050,-0.143853


In [53]:
test.shape

(41129, 305)

In [44]:
train.isnull().sum()

listing_id                     0
review_scores_rating           0
review_scores_accuracy         0
review_scores_cleanliness      0
review_scores_checkin          0
review_scores_communication    0
review_scores_location         0
review_scores_value            0
compound                       0
neg                            0
neu                            0
pos                            0
w2v_0                          0
w2v_1                          0
w2v_2                          0
w2v_3                          0
w2v_4                          0
w2v_5                          0
w2v_6                          0
w2v_7                          0
w2v_8                          0
w2v_9                          0
w2v_10                         0
w2v_11                         0
w2v_12                         0
w2v_13                         0
w2v_14                         0
w2v_15                         0
w2v_16                         0
w2v_17                         0
          

In [42]:
for col in test.columns:
    train[col].fillna(train[col].mean(), inplace=True)
    test[col].fillna(test[col].mean(), inplace=True)

In [49]:
predictors = [col for col in test.columns if col != 'listing_id']

In [59]:
train.to_csv('train_sent_w2v_300.csv', index=False)
test.to_csv('test_sent_w2v_300.csv', index=False)

In [54]:
train.head()

,listing_id,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,compound,neg,...,w2v_290,w2v_291,w2v_292,w2v_293,w2v_294,w2v_295,w2v_296,w2v_297,w2v_298,w2v_299
0,6,93,9.0,9.0,10.0,10.0,9.0,9.0,0.906302,0.013705,...,0.332753,0.015466,0.004644,-0.130289,0.152784,0.173813,0.030352,-0.055120,0.116520,-0.114467
1,109,80,10.0,10.0,6.0,8.0,10.0,8.0,0.479400,0.014000,...,0.437039,0.005597,0.043232,0.009074,0.350516,0.309713,-0.247882,-0.017767,0.375197,-0.212089
2,344,80,10.0,10.0,10.0,10.0,10.0,8.0,0.492650,0.000000,...,0.255970,0.275075,0.055811,-0.073632,0.231418,0.462461,-0.211986,-0.248795,0.444771,-0.333489
3,686,95,9.0,10.0,10.0,10.0,10.0,10.0,0.899540,0.010952,...,0.303829,0.072201,-0.073746,-0.055288,0.148126,0.079536,-0.022140,-0.047929,0.066058,-0.141938
4,930,98,10.0,10.0,10.0,10.0,10.0,10.0,0.883906,0.019000,...,0.288086,-0.146234,-0.023919,-0.132986,0.083889,0.148205,0.197570,-0.094648,0.074119,-0.185301


In [55]:
train.shape

(70894, 312)

In [56]:
pd.read_csv('train.csv').shape

(70895, 8)

In [57]:
test.shape

(41129, 305)

In [58]:
test.head()

,listing_id,compound,neg,neu,pos,w2v_0,w2v_1,w2v_2,w2v_3,w2v_4,...,w2v_290,w2v_291,w2v_292,w2v_293,w2v_294,w2v_295,w2v_296,w2v_297,w2v_298,w2v_299
0,177,0.789695,0.020895,0.655263,0.323816,0.125443,0.099148,0.239083,-0.109373,0.115686,...,0.343390,0.079103,0.099657,-0.108110,0.214957,0.185194,-0.007544,-0.052955,0.061775,-0.084681
1,590,0.887377,0.010110,0.679164,0.308788,0.003294,0.097149,0.235431,-0.051583,0.079655,...,0.371077,0.131271,-0.067424,0.024969,0.142040,0.190251,-0.029581,-0.054145,0.055785,-0.112137
2,958,0.821159,0.014551,0.691268,0.286315,0.215096,0.095965,0.287974,-0.119683,0.054009,...,0.298860,-0.070695,-0.036481,-0.167621,0.161097,0.143639,0.066764,0.019375,-0.000382,-0.147448
3,1788,0.886797,0.002567,0.695867,0.301567,0.229944,0.084868,0.307960,-0.106768,0.125780,...,0.353351,0.056552,0.017254,-0.058394,0.197951,0.115530,0.090405,-0.017649,0.054126,-0.167841
4,2384,0.909629,0.011200,0.706920,0.281907,0.109573,0.115508,0.294353,-0.011445,0.069103,...,0.351657,0.046355,-0.075938,0.044893,0.171532,0.197407,0.021918,-0.060119,0.025050,-0.143853
